In [2]:
import torch
from LSTM import LSTM
import Dataloader

model = LSTM(hidden_dim=100)

#loss_fn = torch.nn.MSELoss(size_average=False)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())#, lr=learning_rate)

num_epochs = 5  


N = 50
N_minibatch_loss = 0.0

dataloader = Dataloader.Dataloader()

inputs,targets = dataloader.load_data('../pa4Data/train.txt')

val_inputs,val_targets = dataloader.load_data('../pa4Data/val.txt')

# Track the loss across training
total_loss = []
avg_minibatch_loss = []
val_losses = []
epochs = []
minibatches = []
val_count=0
save_state_dict=None

for epoch in range(num_epochs): 
    model.clear_hidden() # zero out hidden/memory state
    
    for i in range(len(inputs)):
        model.train()
        
        # Zero out gradient
        optimizer.zero_grad()
        
        outputs = model(inputs[i])
        
        loss = criterion(outputs, targets[i].long())
          
        # Backward pass
        loss.backward()
        
        # Update parameters
        optimizer.step()

        # Add this iteration's loss to the total_loss
        total_loss.append(loss.item())
        N_minibatch_loss += loss.item()
        
        if i % N == 0: 
            # Print the loss averaged over the last N mini-batches    
            N_minibatch_loss /= N
            print('Epoch %d, average minibatch %d loss: %.3f' %
                (epoch + 1, i, N_minibatch_loss))
            
            # Add the averaged loss over N minibatches and reset the counter
            avg_minibatch_loss.append(N_minibatch_loss)
            N_minibatch_loss = 0.0
            
            # validation
            print('...validating...')
            val_loss=0.0
            with torch.no_grad():
                for v in range(len(val_inputs)):
                    val_outputs = model(val_inputs[v])
                    val_loss += criterion(val_outputs, val_targets[v]).item()
            val_loss/=len(val_inputs)
            val_losses.append(val_loss)
            
            epochs.append(epoch+1)
            minibatches.append(i)
            
            print('Epoch %d, average validation loss: %.3f' %
                (epoch + 1, val_loss))
            
            # early stopping
            if len(val_losses)>=2 and val_loss>val_losses[-2]:
                if val_count==0:
                    save_state_dict = model.state_dict()
                val_count+=1
                if val_count==5:
                    break  
            
    print("Finished", epoch + 1, "epochs of training")

if save_state_dict:
    PATH = "./output/{}/best.pt".format(model_name)
    torch.save(save_state_dict, PATH)

    


KeyboardInterrupt: 

In [6]:
import pandas as pd

#train_loss_df = {'epoch':epochs,'minibatch':minibatches,'loss':avg_minibatch_loss}
#val_loss_df = {'epoch':epochs,'minibatch':minibatches,'loss':val_losses}

train_loss_df = pd.DataFrame({'loss':avg_minibatch_loss})
val_loss_df = pd.DataFrame({'loss':val_losses})

model_name="lstm" + str(model.hidden_dim)
val_loss_file = "./output/{}/validation_loss.csv".format(model_name)
train_loss_file = "./output/{}/train_loss.csv".format(model_name)
scores_file = "./output/{}/scores.csv".format(model_name)

train_loss_df.to_csv(train_loss_file)
val_loss_df.to_csv(val_loss_file)

In [7]:
PATH = "./output/{}/lstm.pt".format(model_name)
torch.save(model.state_dict(), PATH)

In [2]:
from train import train 
from LSTM import LSTM
import Dataloader
import torch


model = LSTM(hidden_dim=100)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())#, lr=learning_rate)
EPOCHS = 5
output_dir='lstm100_5epoch'


dataloader = Dataloader.Dataloader()
train_inputs,train_targets = dataloader.load_data('../pa4Data/test.txt')
val_inputs,val_targets = dataloader.load_data('../pa4Data/val.txt')


train(model, criterion, optimizer, train_inputs, train_targets, val_inputs, val_targets, output_dir, num_epochs=EPOCHS)
    
    

Epoch 1, average minibatch 0 loss: 0.090
...validating...
Epoch 1, average validation loss: 4.530
Epoch 1, average minibatch 50 loss: 3.955
...validating...
Epoch 1, average validation loss: 3.622
Epoch 1, average minibatch 100 loss: 3.538
...validating...
Epoch 1, average validation loss: 3.628
Epoch 1, average minibatch 150 loss: 3.553
...validating...
Epoch 1, average validation loss: 3.731
Epoch 1, average minibatch 200 loss: 3.453
...validating...
Epoch 1, average validation loss: 3.555
Epoch 1, average minibatch 250 loss: 3.384
...validating...
Epoch 1, average validation loss: 3.438
Epoch 1, average minibatch 300 loss: 3.199
...validating...
Epoch 1, average validation loss: 3.394
Epoch 1, average minibatch 350 loss: 3.106
...validating...
Epoch 1, average validation loss: 3.266
Epoch 1, average minibatch 400 loss: 2.862
...validating...
Epoch 1, average validation loss: 3.192
Epoch 1, average minibatch 450 loss: 2.775
...validating...
Epoch 1, average validation loss: 3.189
Epo

(        loss
 0   0.090478
 1   3.955264
 2   3.537617
 3   3.553275
 4   3.453483
 5   3.383699
 6   3.199141
 7   3.106257
 8   2.862025
 9   2.775416
 10  2.735421
 11  1.863143
 12  2.636249
 13  2.436724
 14  2.578782
 15  2.224940
 16  0.048423
 17  2.254766
 18  2.106628
 19  2.358689
 20  1.983901
 21  2.075039
 22  1.917606
 23  1.827750
 24  1.682531
 25  1.806865
 26  1.959133
 27  1.198345
 28  1.902451
 29  1.787740
 30  2.083501
 31  1.713251
 32  1.819865
 33  1.641254
 34  1.587984
 35  1.461400
 36  1.604349
 37  1.775952
 38  1.057153
 39  1.738091
 40  1.638457
 41  1.920360
 42  1.590486
 43  1.729009
 44  1.593388
 45  1.476837
 46  1.353311
 47  1.500835
 48  1.667480,         loss
 0   4.530399
 1   3.622205
 2   3.628471
 3   3.730939
 4   3.554993
 5   3.437960
 6   3.394003
 7   3.265923
 8   3.191975
 9   3.189005
 10  3.142303
 11  3.112886
 12  2.908919
 13  2.945286
 14  2.950505
 15  2.988925
 16  2.954124
 17  2.801696
 18  2.850075
 19  2.876199
 20  2

CUDA NOT supported


3.5991346127108526